# Building Model for Predicting Rookie RB Success

## NFL Data Loading

In [1]:
import pandas as pd

with open('rookie_rbs.csv', 'r') as f1:
    rookies = pd.read_csv(f1)
rookies.head()

,Year,No.,Round,Pick,Player,Name,Team,College
0,2023,1,1,8,8,Bijan Robinson,Falcons,Texas
1,2023,2,1,12,12,Jahmyr Gibbs,Lions,Alabama
2,2023,3,2,21,52,Zach Charbonnet,Seahawks,UCLA
3,2023,4,3,8,71,Kendre Miller,Saints,Texas Christian
4,2023,5,3,18,81,Tyjae Spears,Titans,Tulane


In [2]:
rookie_year_mapper = {}
for i, row in rookies.iterrows():
    rookie_year_mapper[row["Name"]] = int(row["Year"])
rookie_year_mapper

{'Bijan Robinson': 2023,
 'Jahmyr Gibbs': 2023,
 'Zach Charbonnet': 2023,
 'Kendre Miller': 2023,
 'Tyjae Spears': 2023,
 "De'Von Achane": 2023,
 'Tank Bigsby': 2023,
 'Roschon Johnson': 2023,
 'Israel Abanikanda': 2023,
 'Chase Brown': 2023,
 'Eric Gray': 2023,
 'Evan Hull': 2023,
 'Chris Rodriguez': 2023,
 'Deuce Vaughn': 2023,
 'Zach Evans': 2023,
 'DeWayne McBride': 2023,
 'Lew Nichols': 2023,
 'Kenny McIntosh': 2023,
 'Breece Hall': 2022,
 'Kenneth Walker': 2022,
 'James Cook': 2022,
 'Rachaad White': 2022,
 'Tyrion Davis-Price': 2022,
 'Brian Robinson': 2022,
 'Dameon Pierce': 2022,
 'Zamir White': 2022,
 'Isaiah Spiller': 2022,
 'Pierre Strong': 2022,
 'Hassan Haskins': 2022,
 'Tyler Allgeier': 2022,
 'Snoop Conner': 2022,
 'Jerome Ford': 2022,
 'Kyren Williams': 2022,
 'Ty Chandler': 2022,
 'Kevin Harris': 2022,
 'Tyler Badie': 2022,
 'Keaontay Ingram': 2022,
 'Trestan Ebner': 2022,
 'Connor Heyward': 2022,
 'Brittain Brown': 2022,
 'Isaih Pacheco': 2022,
 'Zander Horvath': 202

In [3]:
with open('rushing_stats.csv', 'r') as f2:
    nfl_stats = pd.read_csv(f2)
nfl_stats.head()

,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year
0,Ricky Williams,MIA,26,RB,16,16,392,1372,9,68,36.7,45,3.5,85.8,7,2003
1,Jamal Lewis,BAL,24,RB,16,16,387,2066,14,85,45.2,82,5.3,129.1,8,2003
2,Ahman Green,GNB,26,RB,16,16,355,1883,15,100,49.3,98,5.3,117.7,7,2003
3,Deuce McAllister,NOR,25,RB,16,16,351,1641,8,69,40.7,76,4.7,102.6,6,2003
4,Fred Taylor,JAX,27,RB,16,16,345,1572,6,79,46.7,62,4.6,98.3,6,2003


In [4]:
nfl_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6895 entries, 0 to 6894
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   player            6895 non-null   object 
 1   team              6895 non-null   object 
 2   age               6895 non-null   int64  
 3   pos               6895 non-null   object 
 4   g                 6895 non-null   int64  
 5   gs                6895 non-null   int64  
 6   rush_att          6895 non-null   int64  
 7   rush_yds          6895 non-null   int64  
 8   rush_td           6895 non-null   int64  
 9   rush_first_down   6895 non-null   int64  
 10  rush_success      6895 non-null   float64
 11  rush_long         6895 non-null   int64  
 12  rush_yds_per_att  6895 non-null   float64
 13  rush_yds_per_g    6895 non-null   float64
 14  fumbles           6895 non-null   int64  
 15  year              6895 non-null   int64  
dtypes: float64(3), int64(10), object(3)
memory

In [5]:
rookie_rbs_03_23 = list(rookie_year_mapper.keys())
rookie_rbs_03_23

['Bijan Robinson',
 'Jahmyr Gibbs',
 'Zach Charbonnet',
 'Kendre Miller',
 'Tyjae Spears',
 "De'Von Achane",
 'Tank Bigsby',
 'Roschon Johnson',
 'Israel Abanikanda',
 'Chase Brown',
 'Eric Gray',
 'Evan Hull',
 'Chris Rodriguez',
 'Deuce Vaughn',
 'Zach Evans',
 'DeWayne McBride',
 'Lew Nichols',
 'Kenny McIntosh',
 'Breece Hall',
 'Kenneth Walker',
 'James Cook',
 'Rachaad White',
 'Tyrion Davis-Price',
 'Brian Robinson',
 'Dameon Pierce',
 'Zamir White',
 'Isaiah Spiller',
 'Pierre Strong',
 'Hassan Haskins',
 'Tyler Allgeier',
 'Snoop Conner',
 'Jerome Ford',
 'Kyren Williams',
 'Ty Chandler',
 'Kevin Harris',
 'Tyler Badie',
 'Keaontay Ingram',
 'Trestan Ebner',
 'Connor Heyward',
 'Brittain Brown',
 'Isaih Pacheco',
 'Zander Horvath',
 'Najee Harris',
 'Travis Etienne',
 'Javonte Williams',
 'Trey Sermon',
 'Michael Carter',
 'Kene Nwangwu',
 'Rhamondre Stevenson',
 'Chuba Hubbard',
 'Kenneth Gainwell',
 'Ben Mason',
 'Elijah Mitchell',
 'Gary Brightwell',
 'Larry Rountree',
 'Ch

In [6]:
rb_stats = nfl_stats[nfl_stats["player"].isin(rookie_rbs_03_23)].reset_index()
rb_stats

,index,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year
0,43,Onterrio Smith,MIN,23,RB,15,3,107,579,5,29,53.3,47,5.4,38.6,1,2003
1,46,Quentin Griffin,DEN,22,RB,10,1,94,345,0,16,39.4,23,3.7,34.5,3,2003
2,65,Chris Brown,TEN,22,RB,11,0,56,221,0,9,39.3,28,3.9,20.1,1,2003
3,67,Lee Suggs,CLE,23,RB,7,0,56,289,2,14,42.9,78,5.2,41.3,2,2003
4,69,LaBrandon Toefield,JAX,23,RB,16,0,53,212,2,11,49.1,30,4.0,13.3,0,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1708,6819,Chris Evans,CIN,26,RB,8,0,2,12,0,0,0.0,7,6.0,1.5,0,2023
1709,6820,Jake Funk,IND,25,RB,4,0,2,10,0,0,0.0,7,5.0,2.5,0,2023
1710,6856,Connor Heyward,PIT,24,TE,17,7,1,0,0,0,0.0,0,0.0,0.0,0,2023
1711,6859,Evan Hull,IND,23,RB,1,0,1,1,0,0,0.0,1,1.0,1.0,0,2023


We now have our NFL stats from all the players that were drafted in the NFL draft from 2003 to 2023. Now we want to look at how many of these players were successful draft picks in their first 2 years.

## Were these players successful in their first 2 years?

In [7]:
f2_year = []
rl = []
for i, row in rb_stats.iterrows():
    name = str(row["player"])
    year = int(row["year"])
    rookie_year = rookie_year_mapper[name]
    rl.append(rookie_year)
    if year == rookie_year or year == rookie_year+1:
        f2_year.append(1)
    else:
        f2_year.append(0)
rb_stats["rookie_year"] = rl
rb_stats["first_2_years"] = f2_year
rb_stats[(rb_stats["year"] == 2023)].head(25)

,index,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year,rookie_year,first_2_years
1620,6549,Derrick Henry,TEN,29,RB,17,17,280,1167,12,63,46.1,69,4.2,68.6,0,2023,2016,0
1621,6550,Christian McCaffrey,SFO,27,RB,16,16,272,1459,14,83,54.0,72,5.4,91.2,3,2023,2017,0
1622,6551,Rachaad White,TAM,24,RB,17,17,272,990,6,46,40.8,38,3.6,58.2,3,2023,2022,1
1623,6552,Travis Etienne,JAX,24,RB,17,17,267,1008,11,53,43.1,62,3.8,59.3,0,2023,2021,0
1624,6553,Joe Mixon,CIN,27,RB,17,17,257,1034,9,59,49.4,44,4.0,60.8,0,2023,2017,0
1625,6554,Najee Harris,PIT,25,RB,17,17,255,1035,8,51,48.2,25,4.1,60.9,2,2023,2021,0
1626,6555,Tony Pollard,DAL,26,RB,17,17,252,1005,6,56,50.0,31,4.0,59.1,3,2023,2019,0
1627,6556,Saquon Barkley,NYG,26,RB,14,14,247,962,6,51,40.1,36,3.9,68.7,2,2023,2018,0
1628,6557,Chuba Hubbard,CAR,24,RB,17,12,238,902,5,59,47.5,22,3.8,53.1,1,2023,2021,0
1629,6558,James Cook,BUF,24,RB,17,13,237,1122,2,53,55.7,42,4.7,66.0,4,2023,2022,1


In [13]:
f2y_stats = rb_stats[rb_stats["first_2_years"] == 1]
f2y_stats[f2y_stats["year"]==2023]

,index,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year,rookie_year,first_2_years
1622,6551,Rachaad White,TAM,24,RB,17,17,272,990,6,46,40.8,38,3.6,58.2,3,2023,2022,1
1629,6558,James Cook,BUF,24,RB,17,13,237,1122,2,53,55.7,42,4.7,66.0,4,2023,2022,1
1632,6561,Kyren Williams,LAR,23,RB,12,11,228,1144,12,62,59.6,56,5.0,95.3,3,2023,2022,1
1633,6562,Breece Hall,NYJ,22,RB,17,16,223,994,5,40,39.5,83,4.5,58.5,2,2023,2022,1
1637,6567,Bijan Robinson,ATL,21,RB,17,16,214,976,4,50,43.5,38,4.6,57.4,4,2023,2023,1
1639,6571,Jerome Ford,CLE,24,RB,17,12,204,813,4,33,39.7,69,4.0,47.8,1,2023,2022,1
1640,6573,Tyler Allgeier,ATL,23,RB,17,3,186,683,4,39,48.4,31,3.7,40.2,0,2023,2022,1
1643,6576,Jahmyr Gibbs,DET,21,RB,15,3,182,945,10,42,46.7,36,5.2,63.0,2,2023,2023,1
1649,6587,Dameon Pierce,HOU,23,RB,14,7,145,416,2,14,37.9,22,2.9,29.7,0,2023,2022,1
1655,6595,Zach Charbonnet,SEA,22,RB,16,2,108,462,1,32,49.1,23,4.3,28.9,0,2023,2023,1


In [14]:
f2y_stats.describe()

,index,age,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year,rookie_year,first_2_years
count,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.0
mean,3425.424471,22.990937,11.845921,3.734139,85.891239,367.383686,2.444109,18.767372,43.031873,29.660121,3.974622,28.641239,1.157100,2013.395770,2012.919940,1.0
std,1862.653110,1.132618,4.492435,4.969682,84.108731,380.551146,3.142946,19.472708,16.668569,21.772810,1.504312,25.764046,1.471701,5.767338,5.774213,0.0
min,43.000000,21.000000,1.000000,0.000000,1.000000,-4.000000,0.000000,0.000000,0.000000,-4.000000,-4.000000,-1.000000,0.000000,2003.000000,2003.000000,1.0
25%,1864.500000,22.000000,9.000000,0.000000,16.000000,53.250000,0.000000,3.000000,37.500000,13.000000,3.300000,6.700000,0.000000,2008.000000,2008.000000,1.0
50%,3392.500000,23.000000,13.000000,1.000000,57.000000,239.500000,1.000000,12.000000,44.400000,24.000000,4.000000,22.000000,1.000000,2013.000000,2013.000000,1.0
75%,4875.750000,24.000000,16.000000,6.750000,133.000000,568.000000,4.000000,29.000000,50.000000,43.000000,4.700000,44.775000,2.000000,2018.000000,2018.000000,1.0
max,6859.000000,29.000000,17.000000,17.000000,363.000000,2006.000000,18.000000,107.000000,100.000000,92.000000,13.800000,125.400000,9.000000,2023.000000,2023.000000,1.0


Here I must draw an arbitrary line as to who had a successful season and who did not. Looking at the names for this year, it appears to me that the best way to gauge this is through rushing attempts and rushing yards. Aurther I think that the averages for each of these columns actually do a great job of dividing the list into successful and not-successful seasons. I will use this as the benchmark for defining a successful season. I will also add in that they should have at least 2.8 yards per carry as anything below this is pretty ineffecitve. This is about one standard deviation away from the mean. 

In [18]:
min_att = f2y_stats.rush_att.mean()
min_yds = f2y_stats.rush_yds.mean()
min_ypc = 2.8

In [22]:
successful_year = []
for i, row in f2y_stats.iterrows():
    if row["rush_att"] >= min_att and row["rush_yds"] >= min_yds and row["rush_yds_per_att"] >= min_ypc:
        successful_year.append(1)
    else:
        successful_year.append(0)
f2y_stats["successful_year"] = successful_year
f2y_stats

/var/folders/z9/9fp5jhgs2bj9rb5803bf8zmw0000gn/T/ipykernel_71037/3778444948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f2y_stats["successful_year"] = successful_year


,index,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,rush_first_down,rush_success,rush_long,rush_yds_per_att,rush_yds_per_g,fumbles,year,rookie_year,first_2_years,successful_year
0,43,Onterrio Smith,MIN,23,RB,15,3,107,579,5,29,53.3,47,5.4,38.6,1,2003,2003,1,1
1,46,Quentin Griffin,DEN,22,RB,10,1,94,345,0,16,39.4,23,3.7,34.5,3,2003,2003,1,0
2,65,Chris Brown,TEN,22,RB,11,0,56,221,0,9,39.3,28,3.9,20.1,1,2003,2003,1,0
3,67,Lee Suggs,CLE,23,RB,7,0,56,289,2,14,42.9,78,5.2,41.3,2,2003,2003,1,0
4,69,LaBrandon Toefield,JAX,23,RB,16,0,53,212,2,11,49.1,30,4.0,13.3,0,2003,2003,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694,6693,Kevin Harris,NWE,23,RB,4,0,16,65,1,2,50.0,18,4.1,16.3,0,2023,2022,1,0
1701,6727,Zach Evans,LAR,22,RB,10,0,9,19,0,0,0.0,5,2.1,1.9,0,2023,2023,1,0
1704,6754,Tyrion Davis-Price,SFO,23,RB,1,0,6,21,0,1,50.0,15,3.5,21.0,0,2023,2022,1,0
1710,6856,Connor Heyward,PIT,24,TE,17,7,1,0,0,0,0.0,0,0.0,0.0,0,2023,2022,1,0


In [24]:
f2y_stats = pd.merge(f2y_stats, rookies[["Name", "Pick", "Team", "College"]], left_on="player", right_on="Name")
f2y_stats

,index,player,team,age,pos,g,gs,rush_att,rush_yds,rush_td,...,rush_yds_per_g,fumbles,year,rookie_year,first_2_years,successful_year,Name,Pick,Team,College
0,43,Onterrio Smith,MIN,23,RB,15,3,107,579,5,...,38.6,1,2003,2003,1,1,Onterrio Smith,8,Vikings,Oregon
1,364,Onterrio Smith,MIN,24,RB,11,6,124,544,2,...,49.5,2,2004,2003,1,1,Onterrio Smith,8,Vikings,Oregon
2,46,Quentin Griffin,DEN,22,RB,10,1,94,345,0,...,34.5,3,2003,2003,1,0,Quentin Griffin,11,Broncos,Oklahoma
3,378,Quentin Griffin,DEN,23,RB,6,4,85,311,2,...,51.8,3,2004,2003,1,0,Quentin Griffin,11,Broncos,Oklahoma
4,65,Chris Brown,TEN,22,RB,11,0,56,221,0,...,20.1,1,2003,2003,1,0,Chris Brown,29,Titans,Colorado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,6678,Deuce Vaughn,DAL,22,RB,7,0,23,40,0,...,5.7,0,2023,2023,1,0,Deuce Vaughn,35,Cowboys,Kansas State
658,6679,Israel Abanikanda,NYJ,21,RB,6,0,22,70,0,...,11.7,1,2023,2023,1,0,Israel Abanikanda,8,Jets,Pittsburgh
659,6688,Eric Gray,NYG,24,RB,13,0,17,48,0,...,3.7,3,2023,2023,1,0,Eric Gray,37,Giants,Oklahoma
660,6727,Zach Evans,LAR,22,RB,10,0,9,19,0,...,1.9,0,2023,2023,1,0,Zach Evans,38,Rams,Mississippi


In [26]:
player_success = f2y_stats.groupby("player")["successful_year"].max().reset_index()
player_success

,player,successful_year
0,AJ Dillon,1
1,Aaron Brown,0
2,Aaron Jones,1
3,Aaron Ripkowski,0
4,Adimchinobe Echemandu,0
...,...,...
375,Zach Charbonnet,1
376,Zach Evans,0
377,Zack Moss,1
378,Zamir White,1


We now have a list of all the successful and unsuccessful draft picks of the last 20 years for RBs! These will be very helpful for training our model. 

## College Data Loading

In [30]:
with open('rb_combine_stats.csv', 'r') as f3:
    combine_stats = pd.read_csv(f3)
combine_stats = combine_stats[combine_stats["Pos"] == "RB"]
combine_stats = combine_stats.dropna(subset=["Ht", "Wt"])
combine_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 4 to 7356
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               636 non-null    object 
 1   Pos                  636 non-null    object 
 2   School               636 non-null    object 
 3   College              589 non-null    object 
 4   Ht                   636 non-null    object 
 5   Wt                   636 non-null    float64
 6   40yd                 592 non-null    float64
 7   Vertical             530 non-null    float64
 8   Bench                448 non-null    float64
 9   Broad Jump           515 non-null    float64
 10  3Cone                331 non-null    float64
 11  Shuttle              343 non-null    float64
 12  Drafted (tm/rnd/yr)  358 non-null    object 
 13  Player-additional    550 non-null    object 
dtypes: float64(7), object(7)
memory usage: 74.5+ KB


In [31]:
combine_stats = combine_stats.drop(['College', 'Drafted (tm/rnd/yr)', 'Player-additional'], axis=1)

In [32]:
combine_stats.head()

,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
4,B.J. Askew,RB,Michigan,6-2,241.0,4.60,NaN,NaN,NaN,NaN,NaN
16,Santonio Beard,RB,Alabama,6-0,222.0,4.62,31.5,NaN,119.0,NaN,NaN
29,Chris Brown,RB,Colorado,6-3,220.0,4.46,NaN,NaN,NaN,NaN,NaN
41,Kerry Carter,RB,Stanford,6-2,238.0,4.61,36.0,20.0,118.0,7.29,4.39
45,Avon Cobourne,RB,West Virginia,5-7,205.0,4.70,31.5,21.0,112.0,7.21,4.18


In [33]:
import numpy as np

combine_stats['Ht_in'] = combine_stats['Ht'].str.split('-').apply(lambda x: int(x[0]) * 12 + int(x[1]) if isinstance(x, list) else np.nan)

In [34]:
combine_stats

,Player,Pos,School,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Ht_in
4,B.J. Askew,RB,Michigan,6-2,241.0,4.60,NaN,NaN,NaN,NaN,NaN,74
16,Santonio Beard,RB,Alabama,6-0,222.0,4.62,31.5,NaN,119.0,NaN,NaN,72
29,Chris Brown,RB,Colorado,6-3,220.0,4.46,NaN,NaN,NaN,NaN,NaN,75
41,Kerry Carter,RB,Stanford,6-2,238.0,4.61,36.0,20.0,118.0,7.29,4.39,74
45,Avon Cobourne,RB,West Virginia,5-7,205.0,4.70,31.5,21.0,112.0,7.21,4.18,67
...,...,...,...,...,...,...,...,...,...,...,...,...
7295,Jaden Shirden,RB,Monmouth,5-8,187.0,4.45,34.5,NaN,117.0,NaN,NaN,68
7330,Kimani Vidal,RB,Troy,5-8,213.0,4.46,37.5,18.0,120.0,7.01,4.15,68
7345,Michael Wiley,RB,Arizona,5-11,210.0,4.51,33.5,NaN,119.0,NaN,4.43,71
7350,Miyan Williams,RB,Ohio St.,5-9,229.0,NaN,NaN,NaN,NaN,NaN,NaN,69


## Combine Data Manipulation

We will now use PCA analysis to fill in the missing data since many players do not participate in the combine.

In [35]:
from sklearn.preprocessing import StandardScaler

numerical_cols = ['Ht_in', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle']
numerical_data = combine_stats[numerical_cols]

In [36]:
missing_values_count = numerical_data.isnull().sum()

scaler = StandardScaler()

numerical_data_imputed = numerical_data.fillna(numerical_data.mean())
numerical_data_normalized = scaler.fit_transform(numerical_data_imputed)

In [37]:
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import numpy as np

def pca_impute(data, n_components=None):
    if n_components is not None:
        pca = PCA(n_components=n_components)
    else:
        pca = PCA()
    pca.fit(data)
    data_pca_transformed = pca.transform(data)
    data_pca_inversed = pca.inverse_transform(data_pca_transformed)
    return data_pca_inversed

In [40]:
numerical_data_pca_imputed = pca_impute(numerical_data_normalized)
numerical_data_imputed_final = scaler.inverse_transform(numerical_data_pca_imputed)
numerical_data_filled = numerical_data.copy()
numerical_data_filled.loc[:, :] = numerical_data_imputed_final
numerical_data_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 4 to 7356
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ht_in       636 non-null    float64
 1   Wt          636 non-null    float64
 2   40yd        636 non-null    float64
 3   Vertical    636 non-null    float64
 4   Bench       636 non-null    float64
 5   Broad Jump  636 non-null    float64
 6   3Cone       636 non-null    float64
 7   Shuttle     636 non-null    float64
dtypes: float64(8)
memory usage: 44.7 KB


In [41]:
numerical_data_filled["Player"] = combine_stats["Player"]
numerical_data_filled

,Ht_in,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Player
4,74.0,241.0,4.600000,34.364151,19.544643,118.72233,7.068943,4.282828,B.J. Askew
16,72.0,222.0,4.620000,31.500000,19.544643,119.00000,7.068943,4.282828,Santonio Beard
29,75.0,220.0,4.460000,34.364151,19.544643,118.72233,7.068943,4.282828,Chris Brown
41,74.0,238.0,4.610000,36.000000,20.000000,118.00000,7.290000,4.390000,Kerry Carter
45,67.0,205.0,4.700000,31.500000,21.000000,112.00000,7.210000,4.180000,Avon Cobourne
...,...,...,...,...,...,...,...,...,...
7295,68.0,187.0,4.450000,34.500000,19.544643,117.00000,7.068943,4.282828,Jaden Shirden
7330,68.0,213.0,4.460000,37.500000,18.000000,120.00000,7.010000,4.150000,Kimani Vidal
7345,71.0,210.0,4.510000,33.500000,19.544643,119.00000,7.068943,4.430000,Michael Wiley
7350,69.0,229.0,4.549764,34.364151,19.544643,118.72233,7.068943,4.282828,Miyan Williams


In [42]:
combine_model_training = pd.merge(numerical_data_filled, player_success, left_on="Player", right_on="player")
combine_model_training

,Ht_in,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Player,player,successful_year
0,74.0,241.0,4.600000,34.364151,19.544643,118.72233,7.068943,4.282828,B.J. Askew,B.J. Askew,0
1,75.0,220.0,4.460000,34.364151,19.544643,118.72233,7.068943,4.282828,Chris Brown,Chris Brown,1
2,70.0,210.0,4.550000,36.000000,17.000000,115.00000,7.068943,4.282828,Chris Brown,Chris Brown,1
3,73.0,219.0,4.350000,34.364151,27.000000,137.00000,7.068943,4.282828,Justin Fargas,Justin Fargas,0
4,67.0,195.0,4.480000,38.000000,21.000000,123.00000,7.068943,4.080000,Quentin Griffin,Quentin Griffin,0
...,...,...,...,...,...,...,...,...,...,...,...
306,72.0,219.0,4.580000,31.500000,19.544643,122.00000,7.068943,4.282828,Roschon Johnson,Roschon Johnson,0
307,71.0,215.0,4.549764,34.364151,19.544643,118.72233,7.068943,4.282828,Kendre Miller,Kendre Miller,0
308,71.0,215.0,4.460000,37.000000,19.544643,124.00000,7.068943,4.282828,Bijan Robinson,Bijan Robinson,1
309,70.0,201.0,4.549764,39.000000,19.544643,125.00000,7.068943,4.282828,Tyjae Spears,Tyjae Spears,1


In [43]:
combine_model_training.columns

Index(['Ht_in', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone',
       'Shuttle', 'Player', 'player', 'successful_year'],
      dtype='object')

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

training_df = combine_model_training[['Ht_in', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone','Shuttle']]
X_train, X_test, Y_train, Y_test = train_test_split(training_df, combine_model_training["successful_year"], train_size=0.85, test_size=0.15, random_state=5)

In [46]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
model = LogisticRegression()
model.fit(X_train, Y_train)
print(training_df.columns)
print(model.coef_)

Index(['Ht_in', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone',
       'Shuttle'],
      dtype='object')
[[-0.2138349   0.69117054 -0.33327709  0.04032528 -0.16645508  0.17824919
  -0.28410266  0.1243053 ]]


This is very interesting and gives us some insight into some of the features that could cause a running back to have success. We can see that height has a negative coefficient, meaning that as players get taller, their success tends to drop off. The is the opposite with weight, which has the highest coefficient meaning that adding weight will increase your success in the NFL. You can also see that as your 40 Yard dash time goes up, then it negatively impacts you. This is a little confusing as having a higher 40yd datapoint is actually a worse time so the negative coefficient makes senes. 

In [54]:
model.score(X_test, Y_test)

0.6170212765957447